In [13]:
import os
import random
from tqdm import tqdm
from pydub import AudioSegment

In [14]:
def split_audo(audio:str,audio_folder:str,sample_length:int,sample_count:int):
    '''
    Данная функция генерирует из одного большого аудио файла много файлов поменьше, 
    из которых будет составляться датасет. Все сгенерированные аудио попадают в отдельную директорию, 
    имеющую название исходного файла. 
    
    ==Input==
    audio - путь к исходному файлу
    audio_folder - директория в котороую будет осуществляться генерация
    sample_length - длина генерируемого примера в секундах
    sample_count - количество примеров
    '''
    # проверяем/создаем директорию audio_folder/filename/
    if not os.path.exists(audio_folder):
        os.makedirs(audio_folder)
    sub_folder = os.path.basename(audio).split('.')[0]
    sample_folder = os.path.join(audio_folder,sub_folder)
    if not os.path.exists(sample_folder):
        os.makedirs(sample_folder)

    Audio = AudioSegment.from_wav(audio) # считываем аудио
    max = int(Audio.duration_seconds - sample_length - 1) # получаем длину аудио
    for i in range(0,sample_count): # с
        start = random.randint(0, max)
        end = start + sample_length
        sample = Audio[start*1000:end*1000]
        sample_name = sub_folder+f'_{i}_.wav'
        sample_path=os.path.join(sample_folder,sample_name)
        sample.export(sample_path, format="wav")

In [15]:
def split_all_audio(audio_path:str,audio_folder:str,sample_length:int,sample_count:int):
    '''
    Данная функция в цикле применит функцию split_audo к каждому файлу в директории с расширением wav
    ==Input==
    audio_path - директория с исходными файлами
    audio_folder - директория в котороую будет осуществляться генерация
    sample_length - длина генерируемого примера в секундах
    sample_count - количество примеров
    
    '''
    for subdir, dirs, files in os.walk(audio_path):
            print(subdir)
            for file in tqdm(files):
                filepath = subdir + os.sep + file
                if filepath.endswith(".wav"):
                    split_audo(filepath,audio_folder,sample_length,sample_count)

In [19]:
split_all_audio('./original_data','./splited_data_5',5,70)

./original_data


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13/13 [00:11<00:00,  1.16it/s]


In [ ]:
split_all_audio('./original_data','./splited_data_10',10,70)